In [2]:
import pandas as pd
from tqdm import tqdm
import shutil
import numpy as np

In [3]:
trainingdata = pd.read_csv("/home/kyra/Desktop/Data/Jaguar/train.csv") #1157
testdata = pd.read_csv("/home/kyra/Desktop/Data/Jaguar/test.csv") #139
validatadata = pd.read_csv("/home/kyra/Desktop/Data/Jaguar/validate.csv") #95

In [33]:
alldata = pd.concat([trainingdata,testdata,validatadata])

In [30]:
trainingdata.columns

Index(['Frame', 'CaptureID', 'SurveyID', 'StationID', 'Survey.Name',
       'SpeciesID', 'Common', 'Species', 'Date', 'Time', 'DayNight', 'X', 'Y',
       'Elevation', 'Habitat', 'Group1', 'Group2', 'Sex', 'Individuals',
       'StationID.1', 'CamNumber1', 'CamNumber2', 'Independent', 'Code',
       'IndividualSex', 'Image', 'LeftImage', 'MaxMDConfidence', 'MaxCategory',
       'category', 'conf', 'bbox1', 'bbox2', 'bbox3', 'bbox4', 'image_path',
       'crop_rel_path', 'max_conf', 'crop_path', 'UID', 'local_path', 'height',
       'width', 'mask_path', 'pose_path', 'FileName'],
      dtype='object')

In [43]:
trainingdata["Date"].max()

'31.10.2009'

In [40]:
alldata["X"].value_counts()

409697    78
414713    77
407347    75
429883    71
415085    69
          ..
418741     1
405900     1
406345     1
404043     1
385626     1
Name: X, Length: 103, dtype: int64

In [21]:
len(testdata[~testdata["UID"].isin([19,13,12,22])]["CaptureID"].value_counts())

37

In [20]:
trainingdata['LeftImage']

0       0
1       1
2       1
3       0
4       0
       ..
1152    1
1153    0
1154    1
1155    1
1156    0
Name: LeftImage, Length: 1157, dtype: int64

In [29]:
pose = ['Right','Left']

In [43]:
for i, row in tqdm(trainingdata.iterrows()):
    path = row['mask_path']
    posepath = os.path.basename(path)
    
    trainingdata.loc[i,'FileName'] = posepath
    shutil.copy(path,posepath)
    


1157it [00:00, 4381.94it/s]


In [17]:
validatadata = validatadata.drop(columns = ['Unnamed: 0'])

In [18]:
validatadata = validatadata.drop_duplicates(subset=['image_path'])

In [37]:
validatadata["image_path"] = validatadata["image_path"].str.replace('S:/Camera Base MT/','/home/kyra/Desktop/Data/Jaguar/')


In [117]:
trainingdata.to_csv("/home/kyra/Desktop/Data/Jaguar/train.csv",index=False)

In [115]:
testdata.to_csv("/home/kyra/Desktop/Data/Jaguar/test.csv",index=False)

In [113]:
validatadata.to_csv("/home/kyra/Desktop/Data/Jaguar/validate.csv",index=False)

In [92]:
validatadata['FileName']

0         SDZG  125_20140726_134505.png
1     SDZG  125_20140726_134505_001.png
2         SDZG  125_20140726_134506.png
3         SDZG  129_20141009_045933.png
4         SDZG  129_20141009_045934.png
                    ...                
90           Areas 15_20071101_1557.png
91            Areas 3_20070904_2012.png
92           Areas 39_20070922_2219.png
93           Areas 39_20070923_0357.png
94            Areas 6_20071002_1955.png
Name: FileName, Length: 95, dtype: object

In [85]:
import os

segments = []

for root, dirs, files in os.walk("/home/kyra/Desktop/Data/Jaguar/Pose", topdown=False):
    for name in files:
        segments.append(os.path.join(root, name))
        
segments = np.asarray(segments)

In [86]:
segments

array(['/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG 162_20140908_185650.png',
       '/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG  199_20140902_092456.png',
       '/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG 196_20140903_193333_001.png',
       ...,
       '/home/kyra/Desktop/Data/Jaguar/Pose/Left/SDZG  189_20140818_221520_001.png',
       '/home/kyra/Desktop/Data/Jaguar/Pose/Left/R05_REBG57_20161107_003820_11070202.png',
       '/home/kyra/Desktop/Data/Jaguar/Pose/Left/SDZG  129_20141009_045933.png'],
      dtype='<U81')

In [99]:
segments[0]

'Right'

In [116]:
for i,f in enumerate(segments):
    row = trainingdata[trainingdata['FileName'] == os.path.basename(f)]
    if not row.empty:
        r = row.index[0]
        pose = f.split(os.sep)[-2]
       # print(pose)
        trainingdata.loc[r,'pose_path'] = f
        trainingdata.loc[r,'LeftImage'] = 0 if pose == 'Right' else 1

In [112]:
validatadata

,Frame,CaptureID,SurveyID,StationID,Survey.Name,SpeciesID,Common,Species,Date,Time,...,crop_rel_path,max_conf,crop_path,UID,mask_path,local_path,height,width,pose_path,FileName
0,S:/Camera Base MT/Images/14/SDZG 125_20140726...,102353,14,473,Espinoza 2014,2,Jaguar,Panthera onca,26.07.2014,1899-12-30 13:45:42,...,/14/,0.937475,/home/kyra/Desktop/Data/Jaguar/Crops/14/SDZG ...,27,/home/kyra/Desktop/Data/Jaguar/Masked/14/SDZG ...,/home/kyra/Desktop/Data/Jaguar/Images/14/SDZG ...,2448,3264,/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG...,SDZG 125_20140726_134505.png
1,S:/Camera Base MT/Images/14/SDZG 125_20140726...,102354,14,473,Espinoza 2014,2,Jaguar,Panthera onca,26.07.2014,1899-12-30 13:45:42,...,/14/,0.949219,/home/kyra/Desktop/Data/Jaguar/Crops/14/SDZG ...,27,/home/kyra/Desktop/Data/Jaguar/Masked/14/SDZG ...,/home/kyra/Desktop/Data/Jaguar/Images/14/SDZG ...,2448,3264,/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG...,SDZG 125_20140726_134505_001.png
2,S:/Camera Base MT/Images/14/SDZG 125_20140726...,102355,14,473,Espinoza 2014,2,Jaguar,Panthera onca,26.07.2014,1899-12-30 13:45:43,...,/14/,0.957552,/home/kyra/Desktop/Data/Jaguar/Crops/14/SDZG ...,27,/home/kyra/Desktop/Data/Jaguar/Masked/14/SDZG ...,/home/kyra/Desktop/Data/Jaguar/Images/14/SDZG ...,2448,3264,/home/kyra/Desktop/Data/Jaguar/Pose/Right/SDZG...,SDZG 125_20140726_134506.png
3,S:/Camera Base MT/Images/14/SDZG 129_20141009...,74700,14,475,Espinoza 2014,2,Jaguar,Panthera onca,09.10.2014,1899-12-30 04:59:00,...,/14/,0.912568,/home/kyra/Desktop/Data/Jaguar/Crops/14/SDZG ...,27,/home/kyra/Desktop/Data/Jaguar/Masked/14/SDZG ...,/home/kyra/Desktop/Data/Jaguar/Images/14/SDZG ...,2448,3264,/home/kyra/Desktop/Data/Jaguar/Pose/Left/SDZG ...,SDZG 129_20141009_045933.png
4,S:/Camera Base MT/Images/14/SDZG 129_20141009...,74701,14,475,Espinoza 2014,2,Jaguar,Panthera onca,09.10.2014,1899-12-30 04:59:01,...,/14/,0.920845,/home/kyra/Desktop/Data/Jaguar/Crops/14/SDZG ...,27,/home/kyra/Desktop/Data/Jaguar/Masked/14/SDZG ...,/home/kyra/Desktop/Data/Jaguar/Images/14/SDZG ...,2448,3264,/home/kyra/Desktop/Data/Jaguar/Pose/Left/SDZG ...,SDZG 129_20141009_045934.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S:/Camera Base MT/Images/4/Areas 15_20071101_1...,3505,4,110,Los Amigos 2007,2,Jaguar,Panthera onca,01.11.2007,1899-12-30 15:57:00,...,/4/,0.867164,/home/kyra/Desktop/Data/Jaguar/Crops/4/Areas 1...,75,/home/kyra/Desktop/Data/Jaguar/Masked/4/Areas ...,/home/kyra/Desktop/Data/Jaguar/Images/4/Areas ...,1228,1818,/home/kyra/Desktop/Data/Jaguar/Pose/Left/Areas...,Areas 15_20071101_1557.png
91,S:/Camera Base MT/Images/4/Areas 3_20070904_20...,3622,4,145,Los Amigos 2007,2,Jaguar,Panthera onca,04.09.2007,1899-12-30 20:12:00,...,/4/,0.973747,/home/kyra/Desktop/Data/Jaguar/Crops/4/Areas 3...,75,/home/kyra/Desktop/Data/Jaguar/Masked/4/Areas ...,/home/kyra/Desktop/Data/Jaguar/Images/4/Areas ...,1228,1818,/home/kyra/Desktop/Data/Jaguar/Pose/Left/Areas...,Areas 3_20070904_2012.png
92,S:/Camera Base MT/Images/4/Areas 39_20070922_2...,3337,4,142,Los Amigos 2007,2,Jaguar,Panthera onca,22.09.2007,1899-12-30 22:19:00,...,/4/,0.960536,/home/kyra/Desktop/Data/Jaguar/Crops/4/Areas 3...,75,/home/kyra/Desktop/Data/Jaguar/Masked/4/Areas ...,/home/kyra/Desktop/Data/Jaguar/Images/4/Areas ...,1228,1818,/home/kyra/Desktop/Data/Jaguar/Pose/Left/Areas...,Areas 39_20070922_2219.png
93,S:/Camera Base MT/Images/4/Areas 39_20070923_0...,3338,4,142,Los Amigos 2007,2,Jaguar,Panthera onca,23.09.2007,1899-12-30 03:57:00,...,/4/,0.970124,/home/kyra/Desktop/Data/Jaguar/Crops/4/Areas 3...,75,/home/kyra/Desktop/Data/Jaguar/Masked/4/Areas ...,/home/kyra/Desktop/Data/Jaguar/Images/4/Areas ...,1228,1818,/home/kyra/Desktop/Data/Jaguar/Pose/Right/Area...,Areas 39_20070923_0357.png


In [16]:
extra = set(extra) - set(list(validatadata['mask_path']))
len(extra)

0

In [119]:
import shutil
for i, row in tqdm(testdata.iterrows()):
    pose = row['pose_path']
    mask = row['mask_path']
    
    shutil.copy(mask,pose)
    

139it [00:00, 5886.71it/s]


In [52]:
for i in extra:
    os.remove(i)

In [ ]:
from PIL import Image
import os
from tqdm import tqdm

In [ ]:

for i, row in tqdm(testdata.iterrows()):
    path = row['pose_path']
    jpegpath = '/home/kyra/Desktop/Data/Jaguar/Masked/24/' + os.path.splitext(os.path.basename(path))[0] + '.jpg'
    

    im1 = Image.open(path)
    rgb_im = im1.convert('RGB')
    rgb_im.save(jpegpath)
